# Playground 9:  Segmentation workflows for lamin b1 (interphase)

This notebook contains the workflows for lamin B1 (interphase-specific), and serves as a starting point for developing a classic segmentation workflow for your data with shell-like shapes.

----------------------------------------

Cell Structure Observations:

* [Lamin B1](https://www.allencell.org/cell-observations/category/lamin)

----------------------------------------

Key steps of the workflows:

* Min-max intensity normalization / Auto-contrast
* 3D Gaussian smoothing 
* 2D filament filter 
* watershed

In [ ]:
import numpy as np
import os

# package for 3d visualization
from itkwidgets import view                              
from aicssegmentation.core.visual import seg_fluo_side_by_side,  single_fluorescent_view, segmentation_quick_view
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16, 12]

# package for io 
from aicsimageio import AICSImage   
import imageio

# function for core algorithm
from aicssegmentation.core.vessel import filament_2d_wrapper
from aicssegmentation.core.pre_processing_utils import intensity_normalization, image_smoothing_gaussian_3d
from aicssegmentation.core.utils import get_middle_frame, hole_filling, get_3dseed_from_mid_frame
from skimage.morphology import remove_small_objects, watershed, dilation, ball

## Loading the data

In [ ]:
# Update the path to your data

FILE_PATH = '/data/centrosomes/raw-data/'

FILE_NAME = 'NC12_interphase_Slide22_Emb21_Img1.tif'

reader = AICSImage(FILE_PATH + FILE_NAME) 
IMG = reader.data

print(IMG.shape)


## Preview of the image

In [ ]:
#####################
structure_channel = 0
#####################

structure_img = IMG[0, 0, structure_channel,:,:,:]
view(single_fluorescent_view(structure_img))

## Image segmentation

### Step 1: Pre-Processing

About selected algorithms and tuned parameters

* **Intensity normalization**

Parameter for Lamin B1 (interphase specific):  `intensity_scaling_param = [4000]`

We use min-max Normalization with an absolute intensity upper bound.

* **Smoothing** 

3D gaussian smoothing with `gaussian_smoothing_sigma = 1`.

About `gaussian_smoothing_sigma`: The large the value is, the more the image will be smoothed. 

In [ ]:
################################
## PARAMETERS for this step ##
intensity_scaling_param = [4000]
gaussian_smoothing_sigma = 1
################################
# intensity normalization
structure_img = intensity_normalization(structure_img0, scaling_param=intensity_scaling_param)

# smoothing with 2d gaussian filter slice by slice 
structure_img_smooth = image_smoothing_gaussian_3d(structure_img, sigma=gaussian_smoothing_sigma)

In [ ]:
view(single_fluorescent_view(structure_img_smooth))

### Step 2: Core Algorithm

#### apply 2d filament filter on the middle frame 

##### Part 1: get the middle frame:

We support two methods to get middle frame: `method='intensity'` and `method='z'`. `'intensity'` method assumes the number of foreground pixels (estimated by intensity) along z dimension has a unimodal distribution (such as Gaussian). Then, the middle frame is defined as the frame with peak of the distribution along z. `'z'` method simply return the middle z frame. 

Paramete for lamin b1 (interphase-specific): `'method='intensity''`

##### Part 2: apply 2d filament filter on the middle frame

Parameter syntax: `[[scale_1, cutoff_1], [scale_2, cutoff_2], ....]` 
* `scale_x` is set based on the estimated width of your target curvilinear shape. For example, if visually the width of the objects is usually 3~4 pixels, then you may want to set `scale_x` as `1` or something near `1` (like `1.25`). Multiple scales can be used, if you have objects of very different sizes.  
* `cutoff_x` is a threshold applied on the actual filter reponse to get the binary result. Smaller `cutoff_x` may yielf fatter segmentation, while larger `cutoff_x` could be less permisive and yield less objects and slimmer segmentation. 

Parameter for lamin b1 (interphase-specific):  `f2_param = [[0.5, 0.01]]`

In [ ]:
mid_z = get_middle_frame(structure_img_smooth, method='intensity')

In [ ]:
################################
## PARAMETERS for this step ##
f2_param = [[1, 0.01],[2, 0.01],[3, 0.01]]
################################

bw_mid_z = filament_2d_wrapper(structure_img_smooth[mid_z,:,:], f2_param)

In [ ]:
view(segmentation_quick_view(bw_mid_z))

#### Part 3: apply watershed to get the shell

In [ ]:
####################
hole_max = 40000
hole_min = 400
####################

bw_fill_mid_z = hole_filling(bw_mid_z, hole_min, hole_max)
seed = get_3dseed_from_mid_frame(np.logical_xor(bw_fill_mid_z, bw_mid_z), structure_img.shape, mid_z, hole_min)
bw_filled = watershed(structure_img, seed.astype(int), watershed_line=True)>0

# get the shell
bw_shell = np.logical_xor(bw_filled, dilation(bw_filled, selem=ball(1)))

In [ ]:
viewer_bw = view(segmentation_quick_view(bw_shell))
viewer_bw

##### After quickly visualizing the segmentation results, you can also visualize the segmentation and original image side by side
##### You may select an ROI to inspect the details

* Option 1: Easy ROI selection, but NOT recommended if you are using a laptop

You can select an ROI in above visualization ('viewer_bw'); otherwise, the default ROI is the full image

[See this video for How to select ROI](https://www.youtube.com/watch?v=ZO8ey6-tF_0&index=3&list=PL2lHcsoU0YJsh6f8j2vbhg2eEpUnKEWcl)

* Option 2: Manually type in ROI coordinates

Type in the coordinates of upper left corner and lower right corner of the ROI in the form of [Upper_Left_X, Upper_Left_Y, Lower_right_X, Lower_right_Y]. 

In [ ]:
# Option 1:
# view(seg_fluo_side_by_side(structure_img,bw_shell,roi=['ROI',viewer_bw.roi_slice()]))

# Option 2: 
view(seg_fluo_side_by_side(structure_img,bw_shell,roi=['M',[570,370,730,440]]))

### You may also physically save the segmentation results into a ome.tif file

In [ ]:
# define where to save your test segmentations

output_filepath = '/output/test-segmentations/'

if not os.path.isdir(output_filepath):
    os.makedirs(output_filepath)


In [ ]:
# this file will be saved within your docker container volume "output"
# in order to visualize this most easily, you can copy this to your computer using
# docker cp jupyter:/output/ output/ 

output_seg = bw_shell>0
out=output_seg.astype(np.uint8)
out[out>0]=255
imageio.volwrite(output_filepath + FILE_NAME + '-test_seg.tiff', out)
